<a href="https://colab.research.google.com/github/bhoop70233/Classification-Trees-using-CART-with-Gini-Index/blob/main/Classification_Trees_using_CART_with_Gini_Index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import numpy as np
from collections import Counter

class Node:
    def __init__(self, gini, num_samples, num_samples_per_class, predicted_class):
        self.gini = gini
        self.num_samples = num_samples
        self.num_samples_per_class = num_samples_per_class
        self.predicted_class = predicted_class
        self.feature_index = 0
        self.threshold = 0
        self.left = None
        self.right = None

def gini_impurity(y):
    m = len(y)
    return 1.0 - sum((np.sum(y == c) / m) ** 2 for c in np.unique(y))

def best_split(X, y):
    m, n = X.shape
    if m <= 1:
        return None, None

    # Get unique class labels and their counts
    classes, counts = np.unique(y, return_counts=True)
    num_parent = dict(zip(classes, counts))

    best_gini = 1.0 - sum((num / m) ** 2 for num in num_parent.values())
    best_idx, best_thr = None, None

    for idx in range(n):
        # Sort data along selected feature
        thresholds, classes = zip(*sorted(zip(X[:, idx], y)))
        num_left = {c: 0 for c in num_parent}
        num_right = num_parent.copy()
        for i in range(1, m):
            c = classes[i - 1]
            num_left[c] += 1
            num_right[c] -= 1
            gini_left = 1.0 - sum((num_left[x] / i) ** 2 for x in num_left)
            gini_right = 1.0 - sum((num_right[x] / (m - i)) ** 2 for x in num_right)
            gini = (i * gini_left + (m - i) * gini_right) / m
            if thresholds[i] == thresholds[i - 1]:
                continue
            if gini < best_gini:
                best_gini = gini
                best_idx = idx
                best_thr = (thresholds[i] + thresholds[i - 1]) / 2
    return best_idx, best_thr

def grow_tree(X, y, depth=0, max_depth=None):
    num_samples_per_class = [np.sum(y == i) for i in np.unique(y)]
    predicted_class = np.argmax(num_samples_per_class)
    node = Node(
        gini=gini_impurity(y),
        num_samples=len(y),
        num_samples_per_class=num_samples_per_class,
        predicted_class=predicted_class,
    )

    if depth < max_depth:
        idx, thr = best_split(X, y)
        if idx is not None:
            indices_left = X[:, idx] < thr
            X_left, y_left = X[indices_left], y[indices_left]
            X_right, y_right = X[~indices_left], y[~indices_left]
            node.feature_index = idx
            node.threshold = thr
            node.left = grow_tree(X_left, y_left, depth + 1, max_depth)
            node.right = grow_tree(X_right, y_right, depth + 1, max_depth)
    return node

def predict(node, X):
    if node.left is None and node.right is None:
        return node.predicted_class
    if X[node.feature_index] < node.threshold:
        return predict(node.left, X)
    else:
        return predict(node.right, X)

# Example usage
X = np.array([[2.5], [5.1], [3.3], [4.7], [1.8]])
y = np.array([0, 1, 0, 1, 0])

tree = grow_tree(X, y, max_depth=3)

test_point = np.array([4.0])
prediction = predict(tree, test_point)
print(f"Prediction for {test_point}: {prediction}")


Prediction for [4.]: 0
